In [1]:
import torch
from torch import nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        super().__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        self.output = nn.Linear(hidden_layers[-1], output_size)
        self.dropout = nn.Dropout(p=drop_p)
    
    def forward(self, x):
        
        for each in self.hidden_layers:
            x = F.relu(each(x))
            x = self.droput(x)
        x = self.output(x)
        return F.log_softmax(x, dim = 1)
    
def validation(model, testloader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in testloader:
        images = images.resize_(images.size()[0], 784)
        output = model.forward(images)
        test_loss +=criterion(output, labels).item()
        
        ps = torch.exp(output)
        
        equals = (labels.data == ps.max(1)[1])
        accuracy += equals.type_as(torch.FloatTensor().mean())
    return test_loss, accuracy

def train(model, trainloader, testloader, criterion, optimizer, epochs=5, print_eve)